In [38]:
import pandas as pd
import requests

housing_coord = pd.read_csv("/datasets/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")

housing_coord['address'] = housing_coord['block'] + " " + housing_coord['street_name']
housing_coord['address']

0         406 ANG MO KIO AVE 10
1          108 ANG MO KIO AVE 4
2          602 ANG MO KIO AVE 5
3         465 ANG MO KIO AVE 10
4          601 ANG MO KIO AVE 5
                  ...          
145618        511B YISHUN ST 51
145619         760 YISHUN ST 72
145620        344 YISHUN AVE 11
145621       387 YISHUN RING RD
145622       328 YISHUN RING RD
Name: address, Length: 145623, dtype: object

In [44]:
def getcoordinates(address):
    req = requests.get(f'https://developers.onemap.sg/commonapi/search?searchVal={address}&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    resultsdict = eval(req.text)
    if len(resultsdict['results'])>0:
        return resultsdict['results'][0]['LATITUDE'], resultsdict['results'][0]['LONGITUDE']
    else:
        pass

In [45]:
coordinateslist= []
count = 0
failed_count = 0


In [46]:
# addresslist1 = list(housing_coord['address'])[:1000]
# addresslist2 = list(housing_coord['address'])[1000:2000]
# addresslist3 = list(housing_coord['address'])[2000:3000]
# addresslist4 = pd.Series(list(housing_coord['address'])[3000:4000]).drop_duplicates()
# addresslist5 = pd.Series(list(housing_coord['address'])[4000:]).drop_duplicates()
# addresslist6 = list(housing_coord['address'])[5000:6000]
# addresslist7 = list(housing_coord['address'])[6000:7000]
# addresslist8 = list(housing_coord['address'])[7000:8000]
# addresslist9 = list(housing_coord['address'])[8000:9000]
# addresslist10 = list(housing_coord['address'])[9000:9500]

addresslist = pd.Series(list(housing_coord['address'])).drop_duplicates()

addresslist1 = addresslist[:4500]
addresslist2 = addresslist[4500:]

In [50]:
for address in addresslist2:
    try:
        if len(getcoordinates(address))>0:
            count = count + 1
            print('Extracting',count,'out of',len(addresslist2),'addresses')
            temp = getcoordinates(address)
            print(address, temp)
            coordinateslist.append(temp)
    except:
        count = count + 1           
        failed_count = failed_count + 1
        print('Failed to extract',count,'out of',len(addresslist2),'addresses')
        coordinateslist.append(None)
print('Total Number of Addresses With No Coordinates',failed_count)

df_coordinates = pd.DataFrame(coordinateslist)
df_coordinates

Extracting 8912 out of 4910 addresses
216A COMPASSVALE DR ('1.39078747856513', '103.893415096676')
Extracting 8913 out of 4910 addresses
520B TAMPINES CTRL 8 ('1.35726635061626', '103.939665206968')
Extracting 8914 out of 4910 addresses
889C WOODLANDS DR 50 ('1.43781260724005', '103.79240942154')
Extracting 8915 out of 4910 addresses
889A WOODLANDS DR 50 ('1.43756234209627', '103.793320189052')
Extracting 8916 out of 4910 addresses
126 MARSILING RISE ('1.4381547085134', '103.779841066241')
Extracting 8917 out of 4910 addresses
133 MARSILING RISE ('1.43753389486161', '103.778070969184')
Extracting 8918 out of 4910 addresses
664 BUFFALO RD ('1.30613246548155', '103.851112543389')
Extracting 8919 out of 4910 addresses
807C CHOA CHU KANG AVE 1 ('1.37615600248366', '103.745821319559')
Extracting 8920 out of 4910 addresses
254 CHOA CHU KANG AVE 2 ('1.37822238271604', '103.743885220226')
Extracting 8921 out of 4910 addresses
440A CLEMENTI AVE 3 ('1.31600132585021', '103.764185783471')
Extract

,0,1
0,1.36200453938712,103.853879910407
1,1.37094273993861,103.837974822369
2,1.38070883044887,103.835368226602
3,1.3662010408294,103.857200967235
4,1.38104134784496,103.835131744823
...,...,...
9405,1.40229542971451,103.890631891929
9406,1.36313376810873,103.95545857172
9407,1.36368314068034,103.95499248871
9408,1.4194583236401,103.843091718209


In [53]:
df_coordinates = pd.DataFrame(coordinateslist)
df_combined = housing_coord.join(df_coordinates)
df_combined  = df_combined.rename(columns={0:'Latitude', 1:'Longitude'}) 
df_combined

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,Latitude,Longitude
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,406 ANG MO KIO AVE 10,1.36200453938712,103.853879910407
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,108 ANG MO KIO AVE 4,1.37094273993861,103.837974822369
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,602 ANG MO KIO AVE 5,1.38070883044887,103.835368226602
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,465 ANG MO KIO AVE 10,1.3662010408294,103.857200967235
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,601 ANG MO KIO AVE 5,1.38104134784496,103.835131744823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145618,2023-01,YISHUN,5 ROOM,511B,YISHUN ST 51,07 TO 09,113.0,Improved,2017,93 years 07 months,680000.0,511B YISHUN ST 51,NaN,NaN
145619,2023-01,YISHUN,5 ROOM,760,YISHUN ST 72,04 TO 06,122.0,Improved,1987,63 years 03 months,548000.0,760 YISHUN ST 72,NaN,NaN
145620,2023-01,YISHUN,EXECUTIVE,344,YISHUN AVE 11,01 TO 03,145.0,Maisonette,1988,64 years 07 months,755000.0,344 YISHUN AVE 11,NaN,NaN
145621,2023-01,YISHUN,EXECUTIVE,387,YISHUN RING RD,10 TO 12,145.0,Apartment,1988,64 years 06 months,770000.0,387 YISHUN RING RD,NaN,NaN


In [49]:
df_combined.to_csv('housing_part1.csv', index=False)

In [59]:
coordinateslist

[('1.36200453938712', '103.853879910407'),
 ('1.37094273993861', '103.837974822369'),
 ('1.38070883044887', '103.835368226602'),
 ('1.3662010408294', '103.857200967235'),
 ('1.38104134784496', '103.835131744823'),
 ('1.37680678506745', '103.842017962576'),
 ('1.36750371103856', '103.855897952833'),
 ('1.36511907555698', '103.841742483384'),
 ('1.37005500420092', '103.854880566919'),
 ('1.37405846295585', '103.854168170426'),
 ('1.36754924977907', '103.836740628478'),
 ('1.36682360708102', '103.83649123895'),
 ('1.36598198004505', '103.840654039612'),
 ('1.37447378519738', '103.854100515826'),
 ('1.36885444352138', '103.837896404895'),
 ('1.3697435855576', '103.856023138277'),
 ('1.38156171229824', '103.84000226873'),
 ('1.36605027829032', '103.854168309853'),
 ('1.38219867741921', '103.841211533475'),
 ('1.36956344301912', '103.844698910817'),
 ('1.37065487078854', '103.834630215787'),
 ('1.36830996981351', '103.853600305664'),
 ('1.369196965617', '103.841666636086'),
 ('1.370709786879

In [60]:
housing_list = housing_coord['address'].drop_duplicates()
list(housing_list)

['406 ANG MO KIO AVE 10',
 '108 ANG MO KIO AVE 4',
 '602 ANG MO KIO AVE 5',
 '465 ANG MO KIO AVE 10',
 '601 ANG MO KIO AVE 5',
 '150 ANG MO KIO AVE 5',
 '447 ANG MO KIO AVE 10',
 '218 ANG MO KIO AVE 1',
 '571 ANG MO KIO AVE 3',
 '534 ANG MO KIO AVE 10',
 '233 ANG MO KIO AVE 3',
 '235 ANG MO KIO AVE 3',
 '219 ANG MO KIO AVE 1',
 '536 ANG MO KIO AVE 10',
 '230 ANG MO KIO AVE 3',
 '570 ANG MO KIO AVE 3',
 '624 ANG MO KIO AVE 4',
 '441 ANG MO KIO AVE 10',
 '625 ANG MO KIO AVE 9',
 '119 ANG MO KIO AVE 3',
 '255 ANG MO KIO AVE 4',
 '432 ANG MO KIO AVE 10',
 '211 ANG MO KIO AVE 3',
 '584 ANG MO KIO AVE 3',
 '118 ANG MO KIO AVE 4',
 '333 ANG MO KIO AVE 1',
 '256 ANG MO KIO AVE 4',
 '330 ANG MO KIO AVE 1',
 '557 ANG MO KIO AVE 10',
 '302 ANG MO KIO AVE 3',
 '575 ANG MO KIO AVE 10',
 '509 ANG MO KIO AVE 8',
 '472 ANG MO KIO AVE 10',
 '475 ANG MO KIO AVE 10',
 '629 ANG MO KIO AVE 4',
 '546 ANG MO KIO AVE 10',
 '131 ANG MO KIO AVE 3',
 '254 ANG MO KIO AVE 4',
 '470 ANG MO KIO AVE 10',
 '463 ANG MO

In [61]:
final_coord = pd.DataFrame({'housing_street': housing_list, 'coordinateslist': coordinateslist})
final_coord.head()

,housing_street,coordinateslist
0,406 ANG MO KIO AVE 10,"(1.36200453938712, 103.853879910407)"
1,108 ANG MO KIO AVE 4,"(1.37094273993861, 103.837974822369)"
2,602 ANG MO KIO AVE 5,"(1.38070883044887, 103.835368226602)"
3,465 ANG MO KIO AVE 10,"(1.3662010408294, 103.857200967235)"
4,601 ANG MO KIO AVE 5,"(1.38104134784496, 103.835131744823)"


In [65]:
latitude_list, longitude_list = zip(*coordinateslist)

In [67]:
final_coord = pd.DataFrame({'housing_street': housing_list, 'latitude': latitude_list, 'longitude': longitude_list})
final_coord.head()

,housing_street,latitude,longitude
0,406 ANG MO KIO AVE 10,1.36200453938712,103.853879910407
1,108 ANG MO KIO AVE 4,1.37094273993861,103.837974822369
2,602 ANG MO KIO AVE 5,1.38070883044887,103.835368226602
3,465 ANG MO KIO AVE 10,1.3662010408294,103.857200967235
4,601 ANG MO KIO AVE 5,1.38104134784496,103.835131744823


In [68]:
final_coord.to_csv('housing_coordinates.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d10a9bce-ccf4-48fc-b2f9-9f42f99a7043' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>